In [5]:
import os
import matplotlib.pyplot as plt
import traceback
import gzip
import shutil
import tensorflow as tf
import pandas as pd

from tensorflow.keras import layers

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
def create_dir(dir_path):

  # Check whether the specified path exists or not
  isExist = os.path.exists(dir_path)

  if not isExist:
    
    # Create a new directory because it does not exist 
    os.makedirs(dir_path)
    print(f"{dir_path} directory is created!")


root_path = './mnist_images'
train_path = os.path.join(root_path, "train")
test_path = os.path.join(root_path, "test")

create_dir(root_path)
create_dir(train_path)
create_dir(test_path)

In [7]:
train, test= tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)
train_set, train_lables = train
test_set, test_lables = test

In [8]:
def write_dataset_to_files(target_dir_path, X, y):
  df = pd.DataFrame(columns=["file_name", "label"])
  for i_idx, x in enumerate(X):
    img_name = "_".join(["img", str(i_idx)]) + '.jpg'
    img_path = os.path.join(target_dir_path, img_name)
    if i_idx % 100 == 0:
      print(f"{str(i_idx)} image was processed")
    plt.imsave(img_path, x)
    df = df.append({"file_name": img_name, "label": y[i_idx] }, ignore_index=True)
  labels_path = os.path.join(target_dir_path,"labels.csv")
  df.to_csv(labels_path, index=False)
  return labels_path

print('Writing train images to file...')
train_labels_path = write_dataset_to_files(train_path, train_set[:1000], train_lables[:1000])
print('Writing test images to file...')
test_labels_path = write_dataset_to_files(test_path, test_set[:200], test_lables[:200])

Writing train images to file...
0 image was processed
100 image was processed
200 image was processed
300 image was processed
400 image was processed
500 image was processed
600 image was processed
700 image was processed
800 image was processed
900 image was processed
Writing test images to file...
0 image was processed
100 image was processed


In [9]:
train_labels_df = pd.read_csv(train_labels_path)
test_labels_df = pd.read_csv(test_labels_path)

train_labels_df.head()

,file_name,label
0,img_0.jpg,5
1,img_1.jpg,0
2,img_2.jpg,4
3,img_3.jpg,1
4,img_4.jpg,9


In [10]:
train_file_names = train_labels_df["file_name"].values
train_labels = train_labels_df["label"].values

test_file_names = test_labels_df["file_name"].values
test_labels = test_labels_df["label"].values

In [11]:
def read_image(img_file, label):
  image_path = train_path + '/' + img_file
  image = tf.io.read_file(image_path)
  image = tf.image.decode_image(image, channels=1, dtype=tf.float32)
  return image, label

def augment(image, label):
  # do image augmentation here
  return image, label

ds_train = tf.data.Dataset.from_tensor_slices((train_file_names, train_labels)) # similar to zip()

ds_train = ds_train.map(read_image).map(augment).batch(64)
ds_train

2022-02-22 08:53:56.134289: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-22 08:53:56.134331: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-22 08:53:56.134359: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (adrian-Latitude-5491): /proc/driver/nvidia/version does not exist
2022-02-22 08:53:56.134747: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<BatchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [12]:
model = tf.keras.Sequential([
  layers.Input((28,28,1)),
  layers.Conv2D(16, 3, padding="same"),
  layers.Conv2D(32, 3, padding="same"),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(10),           
])

model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = [tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)],
    metrics = ["accuracy"]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 10)                62730     
                                                                 
Total params: 67,530
Trainable params: 67,530
Non-trainable params: 0
____________________________________________________

In [13]:
model.fit(ds_train, epochs=10, verbose=1)

Epoch 1/10
16/16 [==============================] - 1s 25ms/step - loss: 1.8328 - accuracy: 0.5100
Epoch 2/10
16/16 [==============================] - 0s 25ms/step - loss: 0.7263 - accuracy: 0.8000
Epoch 3/10
16/16 [==============================] - 0s 23ms/step - loss: 0.4493 - accuracy: 0.8670
Epoch 4/10
16/16 [==============================] - 0s 23ms/step - loss: 0.3721 - accuracy: 0.8870
Epoch 5/10
16/16 [==============================] - 0s 23ms/step - loss: 0.3204 - accuracy: 0.9130
Epoch 6/10
16/16 [==============================] - 0s 27ms/step - loss: 0.2640 - accuracy: 0.9280
Epoch 7/10
16/16 [==============================] - 0s 24ms/step - loss: 0.2062 - accuracy: 0.9460
Epoch 8/10
16/16 [==============================] - 0s 24ms/step - loss: 0.1753 - accuracy: 0.9520
Epoch 9/10
16/16 [==============================] - 0s 23ms/step - loss: 0.1424 - accuracy: 0.9650
Epoch 10/10
16/16 [==============================] - 0s 27ms/step - loss: 0.1119 - accuracy: 0.9730
